1. Firstly Import the onscale module. The onscale module allows the creation of models using OnScale's SimAPI. SimAPI is used to describe a model which can be ran as a simulation on the OnScale cloud platform.

In [1]:
import onscale as on


2. Define the model within the context of an onscale.Simulation object.  The Simulation object contains all of the informaton which will define the model. Within here, Geometry can be added, Materials assigned and Boundary Conditions applied.

In [2]:
with on.Simulation('Swing Arm', 'None') as sim:

    # Define geometry
    geometry = on.CadFile('swing_arm_2.step')

    # Define material database and materials
    materials = on.CloudMaterials('onscale')
    steel = materials['steel']
    steel >> geometry.parts[0]

    # Define and apply loads
    fixed = on.loads.Fixed()
    fixed >> geometry.parts[0].faces[496]
    fixed >> geometry.parts[0].faces[373]
    force = on.loads.Force(1000, [0, 0, -1])
    force >> geometry.parts[0].faces[340]
    force >> geometry.parts[0].faces[191]

    # Define meshing
    on.meshes.BasicMedium()

    # Define output variables
    on.fields.Displacement()
    on.fields.Stress()
    on.fields.Strain()

3. Import the onscale_client module to allow the models to be submitted to the cloud.  

In [3]:
import onscale_client as cloud

4. Create a Client object to communicate with the OnScale cloud platform

In [4]:
client = cloud.Client()

* Logged in to OnScale platform using account - OnScale UK


5. Submit the simulation previously defined to the cloud.

In [5]:
new_job = client.submit(input_obj=sim, 
                        max_spend=5,
                        job_name="my_simulation")


* Submitting Simulation 
> Generated my_simulation - id : 58a69a22-373d-41e6-9c71-49c45189d0a6
> my_simulation successfully created - id : 58a69a22-373d-41e6-9c71-49c45189d0a6
blob file swing_arm_2.step successfully uploaded
blob id : 933d7162-8152-4ac7-8bd5-aef71559aea2
blob file simulation.py successfully uploaded
blob id : 2899de15-0517-4622-80ea-0db8aba56603
blob file simulationMetadata.json successfully uploaded
blob id : 131a8f03-d903-41ca-84cd-20802d0e2345

* Estimating my_simulation
> Estimate completed successfully

* Estimate Details 
-----------  -------------
RAM          1264
Cores           1
CH Estimate     0.00611111
-----------  -------------
* my_simulation successfully submitted
> job id : 58a69a22-373d-41e6-9c71-49c45189d0a6


6. Poll job status until the job is finished

In [6]:
status_str = new_job.status()
while status_str != 'FINISHED':
    print('\rCurrent Status: [%s]' % status_str, end='')    
    status_str = new_job.status()

Current Status: [QUEUED]

7. Download results once the job is complete

In [7]:
new_job.download_results(os.path.join(os.getcwd(), 'results'))
